# 🚢 Titanic Survival Prediction: Giới thiệu Bài toán & Phân tích Dữ liệu Khám phá (EDA) Chuyên sâu

**Mục tiêu:** Thực hiện EDA chuyên sâu để hiểu sâu sắc các mối tương quan giữa đặc trưng và biến mục tiêu, từ đó xây dựng chiến lược tiền xử lý dữ liệu hiệu quả nhất.

---

## 1. Giới thiệu Bài toán và Đặc trưng Dữ liệu

### 1.1. Bối cảnh & Mục tiêu

Cuộc thi **"Titanic - Machine Learning from Disaster"** là bài toán phân loại nhị phân kinh điển, thách thức người tham gia xây dựng mô hình dự đoán khả năng sống sót ($\text{Survived}=1$) dựa trên dữ liệu hành khách. Thảm họa này nhấn mạnh một nguyên tắc sinh tồn cơ bản: **cơ hội sống sót không đồng đều**, mà bị chi phối bởi các yếu tố xã hội và nhân khẩu học.

### 1.2. Đặc trưng Đầu vào (Features)

| Biến (Variable) | Loại dữ liệu | Định nghĩa | Ghi chú & Ý nghĩa |
| :--- | :--- | :--- | :--- |
| **PassengerId** | Định danh | ID duy nhất. | Không có ý nghĩa dự đoán. |
| **Survived** | Mục tiêu | Trạng thái sống sót. | $1 = \text{Sống sót}, 0 = \text{Tử vong}$. |
| **Pclass** | Định tính (Thứ tự) | Hạng vé. | Proxy cho **Địa vị Kinh tế-Xã hội (SES)**: $1=\text{Upper}, 3=\text{Lower}$. |
| **Sex** | Định tính | Giới tính. | **Đặc trưng dự đoán mạnh mẽ nhất**. |
| **Age** | Định lượng | Tuổi của hành khách. | **Có giá trị thiếu**. Tuổi có thể là số thập phân. |
| **SibSp** | Định lượng | Số lượng Anh chị em/Vợ chồng đi cùng. | |
| **Parch** | Định lượng | Số lượng Cha mẹ/Con cái đi cùng. | |
| **Fare** | Định lượng | Giá vé hành khách trả. | Tương quan mạnh với Pclass. |
| **Cabin** | Định tính | Số hiệu Cabin. | **Thiếu rất nhiều** ($\approx 77\%$). Chữ cái đầu là vị trí boong tàu. |
| **Embarked** | Định tính | Cổng lên tàu. | $C = \text{Cherbourg}, Q = \text{Queenstown}, S = \text{Southampton}$. |

---

# 2. Thiết lập Môi trường và Tổng quan Dữ liệu

In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Thiết lập hiển thị cho Notebook
pd.set_option('display.max_columns', 15)
sns.set_style('whitegrid')
%matplotlib inline 

# Tải dữ liệu
try:
    train_df = pd.read_csv('train.csv')
    print("Tải dữ liệu thành công.")
except FileNotFoundError:
    print("LỖI: Không tìm thấy file dữ liệu. Hãy đảm bảo 'train.csv' nằm trong cùng thư mục với notebook.")

In [ ]:
# Kiểm tra 5 dòng đầu tiên của tập huấn luyện
print('--- 5 Dòng đầu tiên của tập huấn luyện ---')
print(train_df.head())

# Thông tin kiểu dữ liệu và giá trị thiếu
print("\n--- Thông tin kiểu dữ liệu và giá trị thiếu trong tập huấn luyện ---")
train_df.info()

In [ ]:
# Kiểm tra tổng số giá trị thiếu (NaN) trên tập huấn luyện
print("\n--- Tổng số giá trị thiếu (NaN) trên tập huấn luyện ---")
print(train_df.isnull().sum())

**Nhận xét Tổng quan và Vấn đề Dữ liệu:**

Tập huấn luyện (`train.csv`) bao gồm 891 mẫu. Phân tích `info()` và `isnull().sum()` chỉ ra ba cột chính có vấn đề về dữ liệu thiếu:

1.  **`Cabin` (687/891 thiếu):** Với hơn **77%** giá trị bị thiếu, việc điền khuyết số hiệu Cabin cụ thể là không thực tế. Chiến lược phù hợp là chuyển đổi nó thành một biến nhị phân (`Cabin_Known`) để chỉ ra hành khách có thông tin Cabin hay không, vì thông tin Cabin thường liên quan đến vị trí trên tàu và tầng lớp xã hội.
2.  **`Age` (177/891 thiếu):** Đây là một cột quan trọng và không thể bị loại bỏ. Việc điền khuyết cần phải thông minh hơn, ví dụ sử dụng **trung vị** theo nhóm `Pclass` hoặc `Title` (Danh xưng), thay vì trung bình toàn bộ, để bảo toàn mối quan hệ phân bố tuổi-xã hội.
3.  **`Embarked` (2/891 thiếu):** Chỉ thiếu 2 giá trị, có thể dễ dàng điền bằng giá trị **mode** (cổng lên tàu xuất hiện nhiều nhất) mà không gây ảnh hưởng đáng kể đến mô hình.

---

# 3. Phân tích Dữ liệu Khám phá (EDA)

## 3.1. Phân tích Biến Mục tiêu (`Survived`)

In [ ]:
# Tỷ lệ sống sót chung
survival_rate = train_df['Survived'].mean()
print(f"Tỷ lệ sống sót chung: {survival_rate:.2%}")

# Biểu đồ phân bố
plt.figure(figsize=(6, 4))
sns.countplot(x='Survived', data=train_df, palette='viridis')
plt.title('Phân bố Hành khách Sống sót và Tử vong')
plt.xticks([0, 1], ['Tử vong (0)', 'Sống sót (1)'])
plt.show()

**Nhận xét về Phân bố Sống sót:**

Tỷ lệ sống sót chung trong tập dữ liệu là **38.38%**, cho thấy số lượng hành khách tử vong ($61.62\%$) nhiều hơn đáng kể. Mối quan hệ không cân bằng này là điều cần lưu ý. Mục tiêu của mô hình không chỉ là dự đoán chính xác mà còn là tránh thiên vị (bias) quá mức về lớp đa số (tử vong), mặc dù trong bài toán này, độ chính xác (Accuracy) vẫn là thước đo chính của Kaggle.

## 3.2. Mối quan hệ giữa Sống sót, Giới tính và Hạng vé

In [ ]:
plt.figure(figsize=(15, 6))

# Tỷ lệ sống sót theo Giới tính
plt.subplot(1, 2, 1)
sns.barplot(x='Sex', y='Survived', data=train_df, palette='coolwarm')
plt.title('Tỷ lệ Sống sót theo Giới tính')
plt.ylabel('Tỷ lệ Sống sót')

# Tỷ lệ sống sót theo Hạng vé
plt.subplot(1, 2, 2)
sns.barplot(x='Pclass', y='Survived', data=train_df, palette='Spectral')
plt.title('Tỷ lệ Sống sót theo Hạng vé')
plt.ylabel('Tỷ lệ Sống sót')
plt.tight_layout()
plt.show()

# Biểu đồ tương tác
g = sns.FacetGrid(train_df, col='Pclass', height=4, aspect=1.2)
g.map(sns.barplot, 'Sex', 'Survived', palette='viridis', order=['male', 'female'])
g.set_axis_labels('Giới tính', 'Tỷ lệ Sống sót')
g.set_titles('Hạng vé {col_name}')
g.add_legend()
plt.suptitle('Tỷ lệ Sống sót theo Hạng vé và Giới tính', y=1.02, fontsize=16)
plt.show()

**Nhận xét và Giải thích về `Sex` và `Pclass`:**

**Giới tính (`Sex`)** là đặc trưng dự đoán mạnh mẽ nhất: Tỷ lệ sống sót của phụ nữ cao hơn 74% trong khi nam giới chỉ đạt khoảng 19%. Điều này phản ánh rõ ràng chính sách **"Phụ nữ và trẻ em trước"** được ưu tiên trong quá trình sơ tán.

**Hạng vé (`Pclass`)** cũng đóng vai trò quyết định, đại diện cho **Địa vị Kinh tế-Xã hội (SES)**. Hành khách **Hạng nhất** có tỷ lệ sống sót vượt trội, có thể do cabin của họ nằm ở boong trên, gần xuồng cứu sinh hơn và được ưu tiên tiếp cận. Ngược lại, hành khách **Hạng ba** (Pclass=3) chiếm số lượng lớn nhất nhưng có tỷ lệ sống sót thấp nhất.

**Tương tác:** Phân tích tương tác cho thấy **cả hai yếu tố đều quan trọng**. Ngay cả khi có đặc quyền về giới tính, nếu một người phụ nữ ở Hạng ba, tỷ lệ sống sót của họ vẫn thấp hơn đáng kể so với một người phụ nữ ở Hạng nhất. Điều này cho thấy **vị trí và tầng lớp xã hội** có ảnh hưởng đến mức độ ưu tiên sơ tán, ngay cả trong cùng một nhóm giới tính.

## 3.3. Phân tích Tuổi tác (`Age`)

In [ ]:
plt.figure(figsize=(12, 6))
# Biểu đồ mật độ (KDE) để so sánh phân bố Tuổi giữa hai nhóm
sns.kdeplot(train_df[train_df['Survived'] == 1]['Age'].dropna(), label='Sống sót (1)', color='skyblue', fill=True, alpha=0.6, linewidth=2)
sns.kdeplot(train_df[train_df['Survived'] == 0]['Age'].dropna(), label='Tử vong (0)', color='salmon', fill=True, alpha=0.6, linewidth=2)
plt.title('Phân bố Tuổi theo Trạng thái Sống sót')
plt.xlabel('Tuổi')
plt.legend()
plt.show()

**Nhận xét và Giải thích về `Age`:**

Biểu đồ phân bố tuổi chỉ ra một sự khác biệt rõ rệt: **Trẻ em (dưới 10 tuổi)** có đỉnh sống sót cao hơn hẳn, điều này hoàn toàn phù hợp với chính sách "trẻ em trước" đã được áp dụng. Ngược lại, đỉnh tử vong cao nhất tập trung ở nhóm tuổi **20-35**, đây là nhóm chiếm số lượng lớn nhất và chủ yếu là nam giới Hạng ba. Mối tương quan này khẳng định rằng `Age` cần được giữ lại và thậm chí được biến đổi thành các biến phân loại (ví dụ: `Child`, `Young Adult`, `Senior`) để mô hình hóa tốt hơn các ngưỡng tuổi có tỷ lệ sống sót khác nhau.

## 3.4. Phân tích Kích thước Gia đình (`SibSp` & `Parch`)

In [ ]:
# Tạo đặc trưng mới: Kích thước Gia đình (FamilySize = SibSp + Parch + bản thân)
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1

# Trực quan hóa tỷ lệ sống sót theo FamilySize
plt.figure(figsize=(10, 6))
sns.barplot(x='FamilySize', y='Survived', data=train_df, palette='Set2')
plt.title('Tỷ lệ Sống sót theo Kích thước Gia đình')
plt.ylabel('Tỷ lệ Sống sót')
plt.show()

**Nhận xét và Giải thích về `FamilySize`:**

Việc kết hợp `SibSp` và `Parch` thành `FamilySize` mang lại cái nhìn sâu sắc hơn so với việc phân tích từng biến riêng lẻ.

1.  **Người đi một mình (FamilySize=1):** Có tỷ lệ sống sót thấp hơn mức trung bình. Có lẽ những người này ít được ưu tiên cứu giúp hoặc thiếu sự hỗ trợ trong hỗn loạn.
2.  **Gia đình nhỏ (FamilySize=2, 3, 4):** Nhóm này có tỷ lệ sống sót cao nhất. Điều này có ý nghĩa, vì việc sơ tán một gia đình nhỏ (ví dụ: vợ chồng, hoặc vợ chồng và một con) dễ dàng hơn, và các thành viên có động lực bảo vệ nhau.
3.  **Gia đình lớn (FamilySize $\geq 5$):** Tỷ lệ sống sót giảm mạnh. Sơ tán một gia đình quá đông đúc (ví dụ: 6, 7, 8 người) trở nên cực kỳ khó khăn trong điều kiện thiếu xuồng cứu sinh và thời gian hạn chế, dẫn đến việc phải ưu tiên cho các nhóm nhỏ hơn.

Điều này gợi ý rằng, thay vì dùng `SibSp` và `Parch`, việc tạo ra biến **`IsAlone`** (nhị phân, $1$ nếu $FamilySize=1$) hoặc phân loại `FamilySize` thành các nhóm (Nhỏ, Vừa, Lớn) sẽ mang lại sức mạnh dự đoán cao hơn cho mô hình.

---

# 4. Tóm tắt EDA & Chiến lược Tiền xử lý

**Tóm tắt Chiến lược Tiền xử lý Dữ liệu:**

EDA đã xác định **`Sex`** và **`Pclass`** là các đặc trưng quan trọng nhất. Các đặc trưng khác như `Age`, `SibSp`, và `Parch` cần được xử lý và kỹ thuật hóa để tối đa hóa sức mạnh dự đoán.

| Đặc trưng | Vấn đề / Phát hiện | Kế hoạch Hành động |
| :--- | :--- | :--- |
| **Sex** | Yếu tố mạnh nhất. | Mã hóa nhị phân (`male` $\rightarrow 0$, `female` $\rightarrow 1`). |
| **Pclass** | Yếu tố mạnh. | **Mã hóa danh nghĩa** (One-Hot Encoding) vì mặc dù có thứ tự, mối quan hệ không tuyến tính. |
| **Age** | Thiếu 177 giá trị. | **Điền khuyết** bằng trung vị theo `Title` (Danh xưng) để giữ tính liên tục. Sau đó, có thể phân loại thành các nhóm tuổi. |
| **SibSp, Parch** | Dữ liệu thô. | **Kỹ thuật Đặc trưng:** Tạo biến **`FamilySize`** và biến nhị phân **`IsAlone`**. |
| **Embarked** | Thiếu 2 giá trị. | **Điền khuyết** bằng mode. **Mã hóa danh nghĩa** (One-Hot Encoding). |
| **Cabin** | Thiếu $\approx 77\%$. | **Kỹ thuật Đặc trưng:** Tạo biến nhị phân **`Cabin_Known`** (hoặc trích xuất chữ cái đầu tiên). |
| **Name** | Văn bản thô. | **Kỹ thuật Đặc trưng:** Trích xuất **`Title`** (Danh xưng) và sử dụng như một đặc trưng định tính. |

**Ghi chú:** File này dừng ở phần EDA chuyên sâu và chiến lược tiền xử lý, theo yêu cầu. Không thêm phần mô hình hóa hay kết luận tổng quát.